In [ ]:
import itertools as it

from matplotlib import pyplot as plt
from nbmetalog import nbmetalog as nbm
import numpy as np
import pandas as pd
import seaborn as sns
from teeplot import teeplot as tp


In [ ]:
# prints metadata about notebook runtime
nbm.print_metadata()


In [ ]:
df = pd.read_csv("https://osf.io/xd95a/download")
dfdigest = np.bitwise_xor.reduce(
    pd.util.hash_pandas_object(df),
)
print("{:x}".format(dfdigest))
df


In [ ]:
# filter out incomplete sets of replicates
df1 = df[
    df["a"] == "reconstructed-tree"
].groupby(
    [
        'epoch',
        'mut_distn',
        'num_islands',
        'num_niches',
        'p_island_migration',
        'p_niche_invasion',
        'population_size',
        'tournament_size',
        'treatment',
        'a',
        'resolution',
        'subsampling-fraction',
        'trie-postprocess',
    ],
    dropna=False,
).filter(
    lambda x: len(x) == 50,
).reset_index(drop=True)

df1["a"].unique()


In [ ]:
# filter out incomplete sets of replicates
df2 = df[
    df["a"] == "collapsed-phylogeny"
].groupby(
    [
        'epoch',
        'mut_distn',
        'num_islands',
        'num_niches',
        'p_island_migration',
        'p_niche_invasion',
        'population_size',
        'tournament_size',
        'treatment',
        'a',
    ],
    dropna=False,
).filter(
    lambda x: len(x) == 50,
).reset_index(drop=True)

df2["a"].unique()


In [ ]:
df = pd.concat([df1, df2], ignore_index=True)
df


In [ ]:
# phylometrics whittled down to descriptive, non-redundant set
df["colless-like index"] = df["colless_like_index"]
df["mean pairwise distance"] = df["mean_pairwise_distance"]
df["sum pairwise distance"] = df["sum_distance"]
df["mean evolutionary distinctiveness"] = df[
    "mean_evolutionary_distinctiveness"
]

phylometrics = [
    #     'diversity',
    "colless-like index",
    #     'average_depth',
    #     'average_origin_time',
    #     'average_origin_time_normalized',
    #     'max_depth',
    "mean pairwise distance",
    #     'mean_pairwise_distance_branch_only',
    #     'num_active',
    "sum pairwise distance",
    # "num_ancestors",
    #     'num_outside',
    #     'num_roots',
    #     'num_taxa',
    #     'phylogenetic_diversity',
    #     'sum_pairwise_distance',
    #     'sum_pairwise_distance_branch_only',
    #     'total_orgs',
    #     'tree_size',
    #     'variance_pairwise_distance',
    #     'variance_pairwise_distance_branch_only',
    #     'mrca_depth',
    #     'sackin_index',
    "mean evolutionary distinctiveness",
    #     'sum_evolutionary_distinctiveness',
    #     'variance_evolutionary_distinctiveness',
]


In [ ]:
methodological_variables = [
    "resolution",
    "trie-postprocess",
]


In [ ]:
evolutionary_variables = [
    "num_islands",
    "num_niches",
    "tournament_size",
]


In [ ]:
sensitivity_analysis_variables = [
    "epoch",
    "mut_distn",
]


In [ ]:
fixed_variables = [
    "num_generations",  # homogeneous (num generations per epoch)
    "p_island_migration",  # homogeneous
    "population_size",  # homogeneous
    "p_niche_invasion",  # only use 3.0517578125e-08
    "subsampling-fraction",  # only use 1.0
]


In [ ]:
# ensure fixed variables
df = df[
    (df["p_island_migration"] == 0.01)
    & (df["p_niche_invasion"] == 3.0517578125e-08)
    & ((df["subsampling-fraction"] == 1.0) | df["subsampling-fraction"].isna())
].copy()

for fixed_variable in fixed_variables:
    assert len(df[fixed_variable].dropna().unique()) == 1, excluded_variable

df


In [ ]:
# exclude extraneous variable values
df = df[(df["a"] != "consolidated-phylogeny")].copy()

df


In [ ]:
df["num_niches"].unique()


In [ ]:
df["num_islands"].unique()


In [ ]:
df["tournament_size"].unique()


In [ ]:
df["a"].unique()


In [ ]:
df["regime"] = df.apply(
    lambda row: {
        (1, 1, 2): "plain",
        (1, 1, 1): "weak selection",
        (1, 1, 4): "strong selection",
        (1, 4, 2): "ecology",
        (1, 8, 2): "rich ecology",
        (1024, 1, 2): "spatial structure",
    }.get(
        tuple(row[["num_islands", "num_niches", "tournament_size"]]),
        np.nan,  # default
    ),
    axis="columns",
)
df = df.dropna(axis="index", subset=["regime"])
df["regime"].unique()


In [ ]:
df[df["a"] == "collapsed-phylogeny"]["regime"].unique()


In [ ]:
for setup, setup_df in df.groupby(sensitivity_analysis_variables):
    print(f"{setup=}")
    for regime, regime_df in setup_df.groupby(["regime"]):
        ground_truth = regime_df[(regime_df["a"] == "collapsed-phylogeny")][
            "average_origin_time_normalized"
        ].mean()
        print(f"   regime {ground_truth=}")

        for group, group_df in regime_df[
            (regime_df["a"] == "reconstructed-tree")
        ].groupby(["trie-postprocess", "resolution"]):
            err = (
                group_df["average_origin_time_normalized"].mean()
                - ground_truth
            )
            print("      ", group, len(group_df), err)
